In [2]:
import mysql.connector
import json
import time
def get_connection():
    return mysql.connector.connect(
        host="kadman-aurora-db-instance-1.clq6qs624meb.ap-southeast-2.rds.amazonaws.com",
        user="admin",
        password="12345678",
        database="kadmandb",
        port=3306,
        charset='utf8mb4'
    )

def wait_for_db_connection():
    while True:
        try:
            conn = get_connection()
            print("✅ Successfully connected to MySQL database")
            return conn
        except Exception as e:
            print(f"❌ Failed to connect to DB: {e}. Retrying in 1 seconds...")
            time.sleep(1)

In [18]:
con = wait_for_db_connection()
cursor = con.cursor()


✅ Successfully connected to MySQL database


In [ ]:
# Query all vendor data
cursor.execute("ALTER TABLE vendors DROP COLUMN lineID")
rows = cursor.fetchall()
print(rows)

In [16]:
cursor.execute("ALTER TABLE vendors ADD COLUMN UserProfile VARCHAR(20)")
con.commit()
cursor.close()
con.close()

In [19]:
cursor = con.cursor()
cursor.execute("SHOW CREATE TABLE vendors")
result = cursor.fetchone()
print(result[1])  # The actual CREATE TABLE SQL

cursor.close()
con.close()

CREATE TABLE `vendors` (
  `vendorID` int NOT NULL AUTO_INCREMENT,
  `shop_name` varchar(255) NOT NULL,
  `phone_number` varchar(20) DEFAULT NULL,
  `badges` json DEFAULT NULL,
  `LineID` varchar(20) DEFAULT NULL,
  `UserProfile` varchar(20) DEFAULT NULL,
  PRIMARY KEY (`vendorID`)
) ENGINE=InnoDB AUTO_INCREMENT=113 DEFAULT CHARSET=utf8mb3


In [54]:
conn = wait_for_db_connection()

try:
    cursor
except NameError:
    cursor = None

if cursor is not None:
    try:
        # fetch all unread results if any
        cursor.fetchall()
    except Exception:
        # ignore if no results or error
        pass
    cursor.close()
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS waiting_vendors")
print("✅ Table 'waiting_vendors' dropped successfully.")
cursor.close()
conn.commit()
cursor.close()


✅ Successfully connected to MySQL database
✅ Table 'waiting_vendors' dropped successfully.


False

In [55]:
def create_waiting_vendors_table():
    conn = get_connection()
    cursor = conn.cursor()

    create_table_sql = """
    CREATE TABLE IF NOT EXISTS waiting_vendors (
        LineID VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        UserProfile VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        PRIMARY KEY (LineID)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """

    try:
        cursor.execute(create_table_sql)
        print("✅ Table 'waiting_vendors' created successfully.")
    except mysql.connector.Error as err:
        print(f"❌ Error creating table: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
create_waiting_vendors_table()


✅ Table 'waiting_vendors' created successfully.


In [ ]:
def show_all_rows():
    conn = get_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT * FROM waiting_vendors")
        rows = cursor.fetchall()
        if rows:
            print(f"Rows in 'waiting_vendors' ({len(rows)} rows):")
            for row in rows:
                print(row)
        else:
            print("No rows found in 'waiting_vendors' table.")
    except mysql.connector.Error as err:
        print(f"Error fetching rows: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
show_all_rows()

No rows found in 'waiting_vendors' table.


: 